# Xử lý dữ liệu
- Thứ tự thực hiện xử lý dữ liệu:
    - Nhập dữ liệu từ file
    - Convert column
    - Điền các giá trị NA, dựa vào kết quả phân tích
    - Tạo biến giả (dummy variable)
    - Scaling 1 vài dữ liệu và chuyển kiểu dữ liệu về float32
    - Tách tập train, test
    - Đưa vào auto-sklearn và AutoKeras để train, validation, sau đó test bằng test set.

- Những việc cần làm: (trống)

In [1]:
import warnings
import os
from pprint import pprint
import numpy as np
import pandas as pd
import datetime 
import seaborn as sns
import matplotlib.pyplot as plt
import random

import sklearn
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import tensorflow as tf

import autokeras as ak
import autosklearn.regression

import pickle

random.seed(1000)
# OK

In [2]:
# Ignoring future warnings and deprecation warnings so as not to make the notebook full of warnings
warnings.filterwarnings("ignore")
# OK

In [3]:
# Reading the first few lines of the dataset
data = pd.read_csv("VN_housing_dataset.csv")
data.head()
# OK

,Unnamed: 0,Unnamed: 0.1,Ngày,Địa chỉ,Quận,Huyện,Loại hình nhà ở,Giấy tờ pháp lý,Số tầng,Số phòng ngủ,Diện tích,Dài,Rộng,Giá/m2
0,0,0.0,2020-08-05,"Đường Hoàng Quốc Việt, Phường Nghĩa Đô, Quận C...",Quận Cầu Giấy,Phường Nghĩa Đô,"Nhà ngõ, hẻm",Đã có sổ,4,5 phòng,46 m2,NaN,NaN,"86,96 triệu/m2"
1,1,1.0,2020-08-05,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...",Quận Thanh Xuân,Phường Kim Giang,"Nhà mặt phố, mặt tiền",NaN,NaN,3 phòng,37 m2,NaN,NaN,"116,22 triệu/m2"
2,2,2.0,2020-08-05,"phố minh khai, Phường Minh Khai, Quận Hai Bà T...",Quận Hai Bà Trưng,Phường Minh Khai,"Nhà ngõ, hẻm",Đã có sổ,4,4 phòng,40 m2,10 m,4 m,65 triệu/m2
3,3,3.0,2020-08-05,"Đường Võng Thị, Phường Thụy Khuê, Quận Tây Hồ,...",Quận Tây Hồ,Phường Thụy Khuê,"Nhà ngõ, hẻm",Đã có sổ,NaN,6 phòng,51 m2,12.75 m,4 m,100 triệu/m2
4,4,4.0,2020-08-05,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...",Quận Thanh Xuân,Phường Kim Giang,"Nhà ngõ, hẻm",NaN,NaN,4 phòng,36 m2,9 m,4 m,"86,11 triệu/m2"


In [4]:
# The length of the dataset before cleaning and removing outliers
print("The initial length of the dataset is", str(len(data)), "rows.")
data.isna().sum()
# OK

The initial length of the dataset is 82484 rows.


In [5]:
# Đặt tên lại các cột
df_renamed = data.rename(columns = {"Ngày":"date", "Địa chỉ":"address", "Quận":"district", 
                                  "Huyện":"ward", "Loại hình nhà ở":"type_of_housing",
                                 "Giấy tờ pháp lý":"legal_paper", "Số tầng":"num_floors",
                                 "Số phòng ngủ":"num_bed_rooms", "Diện tích":"squared_meter_area",
                                 "Dài":"length_meter", "Rộng":"width_meter", "Giá/m2":"price"})

df_renamed = df_renamed.where(pd.notnull(df_renamed), "NaN")

# Remove houses with "10 plus" floors and bed rooms, since this cannot be exactly quantified
df_renamed = df_renamed[df_renamed['num_floors'] != 'Nhiều hơn 10']
df_renamed = df_renamed[df_renamed['num_bed_rooms'] != 'nhiều hơn 10 phòng']

# Clean columns and convert numerical columns to float type
df_renamed['district'] = df_renamed['district'].str.replace('Quận ','').str.strip()
df_renamed['ward'] = df_renamed['ward'].str.replace('Phường ','').str.strip()
df_renamed['num_floors'] = df_renamed['num_floors'].str.strip().astype(np.float32)
df_renamed['num_bed_rooms'] = df_renamed['num_bed_rooms'].str.replace(' phòng','').str.strip().astype(np.float32)
df_renamed['squared_meter_area'] = df_renamed['squared_meter_area'].str.replace(' m2','').str.strip().astype(np.float32)
df_renamed['length_meter'] = df_renamed['length_meter'].str.replace(' m','').str.strip().astype(np.float32)
df_renamed['width_meter'] = df_renamed['width_meter'].str.replace(' m','').str.strip().astype(np.float32)

# Clean and convert all prices to million/m2 instead of VND/m2 or billion/m2
df_renamed.loc[df_renamed['price'].str.contains(' tỷ/m2', na=False), 'price'] = df_renamed.loc[df_renamed['price'].str.contains(' tỷ/m2', na=False), 'price'].str.replace(' tỷ/m2','').str.replace('.','').str.replace(',','.').astype(np.float32) * 1000
df_renamed.loc[df_renamed['price'].str.contains(' triệu/m2', na=False), 'price'] = df_renamed.loc[df_renamed['price'].str.contains(' triệu/m2', na=False), 'price'].str.replace(' triệu/m2','').str.replace('.','').str.replace(',','.').astype(np.float32)
df_renamed.loc[df_renamed['price'].str.contains(' đ/m2', na=False), 'price'] = df_renamed.loc[df_renamed['price'].str.contains(' đ/m2', na=False), 'price'].str.replace(' đ/m2','').str.replace('.','').str.replace(',','.').astype(np.float32) * 0.000001

df_renamed.head()
df_renamed.to_csv("full_data.csv", index = False)

In [6]:
def filling_na(df):
  for index, row in df.iterrows():
    if pd.isna(row["num_floors"]):
      df.at[index, "num_floors"] = random.choice([4, 5])

    if pd.isna(row["squared_meter_area"]):
      continue
    elif not pd.isna(row["length_meter"]) and pd.isna(row["width_meter"]):
      df.at[index, "width_meter"] = row["squared_meter_area"] / row["length_meter"]
    elif pd.isna(row["length_meter"]) and not pd.isna(row["width_meter"]):
      df.at[index, "length_meter"] = row["squared_meter_area"] / row["width_meter"]
    else:
      continue

  df = df.drop("Unnamed: 0", axis = 1)
  df = df.drop("Unnamed: 0.1", axis = 1)
  df = df.drop(["date", "address"], axis = 1)
  # Xóa bỏ một số giá trị bất thường
  df.drop(df[df['price'] > 400].index, inplace = True)  
  df.drop(df[df['price'] < 1].index, inplace = True)
  df.drop(df[df['num_floors'] > 10].index, inplace = True)
  df.drop(df[abs(df['squared_meter_area'] - df["length_meter"]*df["width_meter"]) > 50].index, inplace = True)
  df.drop(df[df["legal_paper"] == "NaN"].index, inplace=True)
  # Loại bỏ một số cột.
  df = df[["district", "ward", "type_of_housing", "legal_paper", "price"]]


  print(df.isna().sum())
  df = df.dropna()
  df = df.reset_index()
  return df
    
df_renamed = filling_na(df_renamed)
X_train = df_renamed
y_train = X_train.pop("price")
X_train.assign(price=y_train).to_csv("train_data.csv", index=False)
for col in ["district", "ward", "type_of_housing", "legal_paper"]:
    X_train[col] = X_train[col].astype('category')

X_train["index"] = X_train["index"].astype('float32')
y_train = y_train.astype("float32")

print(X_train.dtypes)
# The length of the dataset after dropping null values

print("The length of the dataset after dropping null values is", str(len(df_renamed)), "rows.")
df_renamed.head()

district           0
ward               0
type_of_housing    0
legal_paper        0
price              0
dtype: int64
index               float32
district           category
ward               category
type_of_housing    category
legal_paper        category
dtype: object
The length of the dataset after dropping null values is 51210 rows.


,index,district,ward,type_of_housing,legal_paper
0,0.0,Cầu Giấy,Nghĩa Đô,"Nhà ngõ, hẻm",Đã có sổ
1,2.0,Hai Bà Trưng,Minh Khai,"Nhà ngõ, hẻm",Đã có sổ
2,3.0,Tây Hồ,Thụy Khuê,"Nhà ngõ, hẻm",Đã có sổ
3,7.0,Hai Bà Trưng,Đống Mác,"Nhà mặt phố, mặt tiền",Đã có sổ
4,9.0,Hà Đông,Văn Quán,"Nhà ngõ, hẻm",Đã có sổ


In [7]:
# Sanity check to see if all train and test arrays have correct dimensions
if X_train.shape[0] == y_train.shape[0]:
    print("All train and test sets have correct dimensions.")

All train and test sets have correct dimensions.


# Tìm mô hình phù hợp bằng auto-sklearn

In [8]:
# Dùng Auto-keras
# Initialize the structured data regressor.
reg = ak.StructuredDataRegressor(overwrite=True, max_trials=60)
reg.fit(X_train, y_train ,epochs=12)
model = reg.export_model()
model.save("model_autokeras", save_format="tf")

Trial 60 Complete [00h 03m 27s]
val_loss: 1736.115478515625

Best val_loss So Far: 1707.8848876953125
Total elapsed time: 03h 44m 33s
INFO:tensorflow:Oracle triggered exit
Epoch 1/12
1601/1601 [==============================] - 8s 5ms/step - loss: 2658.5254 - mean_squared_error: 2658.5254
Epoch 2/12
1601/1601 [==============================] - 7s 5ms/step - loss: 2205.8882 - mean_squared_error: 2205.8882
Epoch 3/12
1601/1601 [==============================] - 9s 5ms/step - loss: 2074.5916 - mean_squared_error: 2074.5916
Epoch 4/12
1601/1601 [==============================] - 35s 22ms/step - loss: 1953.2252 - mean_squared_error: 1953.2252
Epoch 5/12
1601/1601 [==============================] - 38s 24ms/step - loss: 1892.1023 - mean_squared_error: 1892.1023
Epoch 6/12
1601/1601 [==============================] - 38s 24ms/step - loss: 1855.9021 - mean_squared_error: 1855.9021
Epoch 7/12
1601/1601 [==============================] - 37s 23ms/step - loss: 1806.4681 - mean_squared_error: 1806

In [9]:
from keras.saving.save import load_model
import autokeras as ak
import numpy as np

loaded_model = load_model("model_autokeras", custom_objects=ak.CUSTOM_OBJECTS)
test_predictions = loaded_model.predict(np.array([[5691, "Đống Đa", "Trung Liệt", "Nhà ngõ, hẻm", "Đã có sổ", 5.0, 4.0, 45.0]]))

ValueError: ValueError: in user code:

    File "/opt/python/envs/default/lib/python3.8/site-packages/keras/engine/training.py", line 1801, in predict_function  *
        return step_function(self, iterator)
    File "/opt/python/envs/default/lib/python3.8/site-packages/keras/engine/training.py", line 1790, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/python/envs/default/lib/python3.8/site-packages/keras/engine/training.py", line 1783, in run_step  **
        outputs = model.predict_step(data)
    File "/opt/python/envs/default/lib/python3.8/site-packages/keras/engine/training.py", line 1751, in predict_step
        return self(x, training=False)
    File "/opt/python/envs/default/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/opt/python/envs/default/lib/python3.8/site-packages/keras/engine/input_spec.py", line 264, in a

Kết quả train model bằng AutoKeras
- 